In [1]:
!pip install Plotly==4.12
!pip install dash
!pip install dash-html-components
!pip install dash-core-components
!pip install dash-table

     |████████████████████████████████| 13.1MB 306kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 81kB 3.5MB/s 
     |████████████████████████████████| 1.0MB 6.9MB/s 
     |████████████████████████████████| 3.5MB 14.4MB/s 
     |████████████████████████████████| 194kB 45.3MB/s 
     |████████████████████████████████| 1.8MB 33.1MB/s 
     |████████████████████████████████| 358kB 36.3MB/s 
  Created wheel for dash: filename=dash-1.19.0-cp37-none-any.whl size=84011 sha256=c60609bfef278067f280b6e63eb6b600062acfa895641cfbd9f79ea6ab6f6aaf
  Stored in directory: /root/.cache/pip/wheels/f7/50/a7/a230ff7f503b10120bff18c2524a375bb85a61ce6b519c8a77
  Created wheel for flask-compress: filename=Flask_Compress-1.9.0-cp37-none-any.whl size=7259 sha256=0515d030a0f17b0218cd7c1d208d6556b4a46401df7114ea9cdbea6195e931b8
  Stored in directory: /root/.cache/pip/wheels/0d/56/7d/e0c50541e5421b8

In [2]:
import pandas as pd 
import zipfile
import matplotlib

In [3]:
import numpy as np
import pandas as pd
import plotly.express as px

In [4]:
# Pacotes para criação de processos para suportar o serviço URL externo do Ngrok
import os.path
import sys, json
import requests
import subprocess

# Pacotes de tratamento de URL
from requests.exceptions import RequestException
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

from collections import namedtuple

In [5]:
# Definição da função de Download do Ngrok
def download_ngrok():
    if not os.path.isfile('ngrok'):
        !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
        !unzip -o ngrok-stable-linux-amd64.zip
    pass

In [6]:
# Criação de um Túnel entre uma aplicação no Colab e uma URL externa no Ngrok
# Função para pegar a referência nesse túnel

Response = namedtuple('Response', ['url', 'error'])

def get_tunnel():
    try:
        Tunnel = subprocess.Popen(['./ngrok','http','8050'])

        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)

        res = session.get('http://localhost:4040/api/tunnels')
        res.raise_for_status()

        tunnel_str = res.text
        tunnel_cfg = json.loads(tunnel_str)
        tunnel_url = tunnel_cfg['tunnels'][0]['public_url']

        return Response(url=tunnel_url, error=None)
    except RequestException as e:
        return Response(url=None, error=str(e))

In [7]:
download_ngrok()

--2021-03-22 03:06:27--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.209.148.13, 54.159.124.229, 3.216.44.25, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.209.148.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  14.3MB/s    in 0.9s    

2021-03-22 03:06:28 (14.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [8]:
pd.options.plotting.backend = 'plotly'

In [109]:
# Geração do programa exemplo_dash_1.py
%%writefile exemplo_dash_1.py
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

#testando 

microdados_RN= pd.read_csv("/content/drive/MyDrive/Teste/microdados_RN.csv")

microdados_RN = microdados_RN.rename({'Q006': 'Renda Familiar','NO_MUNICIPIO_PROVA': 'Município da Prova','NU_NOTA_MT':'Nota de Matemática','NU_NOTA_CN':'Nota Ciências da Natureza'
,'NU_NOTA_CH':'Nota Ciências Humanas', 'NU_NOTA_LC' : 'Nota Linguagens','NU_NOTA_REDACAO':'Nota da Redação','MEDIA_NOTAS':'Média das Notas'
, 'TIPOS_DE_ESCOLA':'Tipo de Escola', 'NU_IDADE': 'Idade dos Candidatos'}, axis = 1)

microdados_RN = microdados_RN.dropna(subset=['Renda Familiar','Nota Ciências da Natureza','Nota Ciências Humanas','Nota Linguagens','Nota de Matemática','Nota da Redação'])
col = microdados_RN.loc[:,"Nota Ciências da Natureza":"Nota da Redação"]
microdados_RN['Média das Notas'] = col.mean(axis=1)


#cidades

colunas_selecionadas_cidades = ['Município da Prova','Nota Ciências da Natureza','Nota Ciências Humanas','Nota Linguagens','Nota de Matemática','Nota da Redação','Média das Notas']
cidades = microdados_RN.filter(items=colunas_selecionadas_cidades)
cidades = cidades.dropna()
cidades.value_counts('Município da Prova')

cidades = cidades.groupby('Município da Prova').mean()
fig1 = px.line(cidades, title='Notas por Cidades', labels=dict(value="Notas", variable="Variável"))

#renda

colunas_selecionadas = ['Renda Familiar','Média das Notas']


renda = microdados_RN.filter(items=colunas_selecionadas)
renda = renda.dropna()
renda = renda.groupby('Renda Familiar').mean()

fig2 = px.area(renda, title='Notas por Renda', labels=dict(value="Notas", variable="Variável"))


#tipo de escola
colunas_selecionadas_escola = ['TP_DEPENDENCIA_ADM_ESC','Média das Notas']
dicionario_escola={1.0:'Federal',2.0:'Estadual',3.0:'Municipal',4.0:'Privada'}
tipo_escola = microdados_RN.filter(items=colunas_selecionadas_escola)
tipo_escola = tipo_escola.dropna()
tipo_escola['Tipo de Escola'] = [dicionario_escola[resp] for resp in tipo_escola.TP_DEPENDENCIA_ADM_ESC]

fig3 = px.histogram(tipo_escola['Tipo de Escola'], title='Quantidade de Escolas por Tipo de Administração', labels=dict(count="Quantidade",value="Tipo de Escola", variable="Variável"))

#nota por tipo de escola

#tipo_escola = tipo_escola.filter(items=['Tipo de Escola','Média das Notas']).groupby('Tipo de Escola').mean()
#fig4 = px.histogram(tipo_escola['Tipo de Escola'], x = 'Tipo de Escola', title='Notas por Tipo de Administração')



#idade


idade_hist = microdados_RN['Idade dos Candidatos'].dropna()
fig5 = px.histogram(idade_hist,title='Histograma da idade dos candidatos',labels=dict(value="Idade dos Candidatos", count="Idade dos Candidatos", variable="Variável"))

#Notas por Idade

colunas_selecionadas_idade = ['Idade dos Candidatos','Média das Notas']
idade = microdados_RN.filter(items=colunas_selecionadas_idade)
idade = idade.dropna()
idade = idade.groupby('Idade dos Candidatos').mean()

fig6 = px.line(idade, title='Média de notas por faixa etária',labels=dict(value="Média", variable="Variável"))


app.layout = html.Div(children=[
    html.H1(children='Análise de Dados do Enem 2019'),

    html.Div(children='''
        Descrição: Análise do desempenho dos estudantes do Rio Grande do Norte no Enem 2019.
    '''),
   

    html.Div(
        dcc.Graph(id='example-graph1',figure=fig1)
     ),

      
    html.Div(
        dcc.Graph(
            id='example-graph2', figure=fig2 )
     ),  
    
      html.Div(
        dcc.Graph(
            id='example-graph3', figure=fig3 )
    ),
      
      html.Div(
        dcc.Graph(
            id='example-graph5', figure=fig5 )
    ),

    html.Div(
        dcc.Graph(
            id='example-graph6', figure=fig6 )
    )
    
])

if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting exemplo_dash_1.py


In [110]:
retorno = get_tunnel()
print(retorno)  # Para podermos saber o endereço da URL da Aplicação
!python exemplo_dash_1.py

Response(url='https://5f68403afcd5.ngrok.io', error=None)
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "exemplo_dash_1" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
